In [1]:
import os
import json
import re
from io import BytesIO
from pathlib import Path

import numpy as np
import pandas as pd
from google.cloud import storage

/Users/loso/.pyenv/versions/3.10.6/envs/inkling/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
def load_environment(env_path: str = ".env") -> tuple[str, str]:
    """Load GCP credentials and bucket name from a .env file (supports running from notebooks/)."""
    env_file = Path(env_path)

    # If running from notebooks/, fall back to parent .env
    if not env_file.is_file() and env_path == ".env":
        parent_env = Path.cwd().parent / ".env"
        if parent_env.is_file():
            env_file = parent_env

    if not env_file.is_file():
        raise FileNotFoundError(f".env file not found at {env_file}")

    for line in env_file.read_text().splitlines():
        stripped = line.strip()
        if not stripped or stripped.startswith("#") or "=" not in stripped:
            continue
        key, value = stripped.split("=", 1)
        key = key.strip()
        value = value.strip()
        if key in {"GOOGLE_APPLICATION_CREDENTIALS", "GCP_BUCKET_NAME"}:
            os.environ[key] = value

    credentials_path = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")
    bucket_name = os.environ.get("GCP_BUCKET_NAME")

    if not credentials_path or not bucket_name:
        raise EnvironmentError(
            "GOOGLE_APPLICATION_CREDENTIALS and GCP_BUCKET_NAME must be set in the .env file."
        )

    credentials_file = Path(credentials_path)
    if not credentials_file.is_file():
        raise FileNotFoundError(f"Credentials file not found at {credentials_file}")

    print(f"Loaded credentials from: {credentials_file}")
    print(f"Using bucket: {bucket_name}")
    return str(credentials_file), bucket_name


In [3]:
def load_emg_dataframe_from_csv(env_path: str = ".env") -> pd.DataFrame:
    """Download the EMG CSV from GCS and return the raw DataFrame."""
    _, bucket_name = load_environment(env_path=env_path)

    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob_path = "EMG-nature/Clean_df/emg_trial_level_df.csv"

    print(f"Accessing blob: {blob_path}")
    blob = bucket.blob(blob_path)

    if not blob.exists(client=client):
        raise FileNotFoundError(f"Blob {blob_path} not found in bucket {bucket_name}")

    data_bytes = blob.download_as_bytes()
    print(f"Downloaded {len(data_bytes)} bytes from GCS")

    df = pd.read_csv(BytesIO(data_bytes))
    print(f"Loaded DataFrame with shape: {df.shape}")
    return df


In [4]:
def parse_signal_cell(cell: str | np.ndarray | list | tuple) -> np.ndarray:
    """Parse a corrupted/loosely formatted signal into a (16, N) float32 array."""
    # If already array-like, trust and reshape as needed.
    if isinstance(cell, (np.ndarray, list, tuple)):
        arr = np.asarray(cell, dtype=np.float32)
        if arr.shape == (16, 10000):
            return arr
        if arr.shape == (10000, 16):
            return arr.T
        if arr.ndim == 1 and arr.size == 160000:
            return arr.reshape(16, 10000)
        if arr.ndim == 2 and arr.shape[0] == 16:
            return arr

    raw = str(cell)
    tokens = re.findall(r"[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?", raw)
    if not tokens:
        raise ValueError(f"No numeric tokens found in signal cell: {raw[:120]}")

    numbers = np.asarray([float(t) for t in tokens], dtype=np.float32)
    if numbers.size < 16:
        raise ValueError(f"Too few numeric tokens ({numbers.size}); sample: {raw[:120]}")

    usable = (numbers.size // 16) * 16
    numbers = numbers[:usable]
    channel_len = usable // 16
    if channel_len == 0:
        raise ValueError(f"Not enough data to form 16 channels; tokens={numbers.size}")

    arr = numbers.reshape(16, channel_len)
    return arr


def fix_length(arr: np.ndarray, target_len: int = 10000) -> np.ndarray:
    """Trim or pad signals to exactly (16, target_len)."""
    if arr.ndim != 2 or arr.shape[0] != 16:
        raise ValueError(f"Signal must have shape (16, N); got {arr.shape}")

    current_len = arr.shape[1]
    if current_len == target_len:
        return arr
    if current_len < target_len:
        pad_width = target_len - current_len
        return np.pad(arr, ((0, 0), (0, pad_width)), mode="constant")
    return arr[:, :target_len]


In [5]:
emg_df = load_emg_dataframe_from_csv()

emg_df["signal"] = emg_df["signal"].apply(parse_signal_cell)
emg_df["signal_fixed"] = emg_df["signal"].apply(fix_length)

print(emg_df["signal_fixed"].apply(lambda x: x.shape).value_counts())
print(emg_df.head())
print(emg_df.shape)


Loaded credentials from: /Users/loso/code/hildieleyser/inkling/keys/inkling-479911-b140632c44c5.json
Using bucket: inkling-ssvep-emg
Accessing blob: EMG-nature/Clean_df/emg_trial_level_df.csv
Downloaded 3078852 bytes from GCS
Loaded DataFrame with shape: (4800, 8)
signal_fixed
(16, 10000)    4800
Name: count, dtype: int64
   Unnamed: 0  participant  day  block  trial_id  position  grasp  \
0           0            1    1      1         1         2      3   
1           1            1    1      1         2         2      3   
2           2            1    1      1         3         2      3   
3           3            1    1      1         4         2      3   
4           4            1    1      1         5         2      3   

                                              signal  \
0  [[3.763498e-05, 1.9842508e-05], [9.071698e-06,...   
1  [[1.0537988e-05, 1.153949e-05], [1.18090165e-0...   
2  [[1.6977565e-05, 1.9937088e-05], [2.1830994e-0...   
3  [[3.6807487e-06, 3.2587977e-06], [

In [6]:
emg_df = emg_df.drop(columns = "Unnamed: 0")

In [7]:
emg_df['signal_fixed'][0]

array([[ 3.76349817e-05,  1.98425078e-05,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 9.07169760e-06, -7.89036676e-06,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.49826537e-05, -1.43748375e-05,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-1.29005557e-05, -9.94938546e-06,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.96866675e-06, -1.09153243e-05,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.30936751e-05, -1.76599133e-05,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

In [8]:
emg_df['signal_fixed'].apply(lambda x: x.shape).value_counts()

signal_fixed
(16, 10000)    4800
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
# emg_df: trial-level df, columns:
# ['participant', 'day', 'block', 'trial_id', 'position', 'grasp', 'signal_fixed']

# Build X from your fixed-length trial signals
X = np.stack(emg_df['signal_fixed'].values)  # shape: (n_trials, 16, T)
X = np.expand_dims(X, axis=-1)               # shape: (n_trials, 16, T, 1)

REST_GRASP_ID = 6  # adjust if needed
y = (emg_df['grasp'].values != REST_GRASP_ID).astype(int)  # 0=rest, 1=active

# We'll split using indices so we can map back to emg_df easily
indices = np.arange(len(emg_df))

train_idx, test_idx = train_test_split(
    indices,
    test_size=0.30,
    stratify=emg_df['grasp'].values,  # stratify by 6-grasp labels
    random_state=42
)

# Slice X and y with indices
X_train = X[train_idx]
X_test  = X[test_idx]
y_train = y[train_idx]
y_test  = y[test_idx]

In [10]:
train_grasps = emg_df['grasp'].values[train_idx]

In [11]:
test_grasps  = emg_df['grasp'].values[test_idx]

In [12]:
print(pd.Series(test_grasps).value_counts().sort_index())

1    240
2    240
3    240
4    240
5    240
6    240
Name: count, dtype: int64


In [13]:
X_train.shape

(3360, 16, 10000, 1)

In [48]:
import tensorflow as tf
from tensorflow.keras import Sequential, Input, layers
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers

In [24]:
es = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True)

In [22]:
X_train.shape[1:]

(16, 10000, 1)

In [23]:
X_train.shape

(3360, 16, 10000, 1)

In [ ]:
# look at a rest wave, look at the other waves (see if the rest wave is different)(i.e. plot it, and check the difference between a none-wave)
#plot all 16 channels (for each wave) in one plot to compare all the waves

#look into function to save training progress

In [29]:
# Infer input shape from X_train
input_shape = X_train.shape[1:]   # (16, T, 1)

#first model score 0.83% accuracy (=baseline, 5/6)

model = Sequential()
model.add(layers.Input(shape=input_shape))

# Conv block 1
model.add(layers.Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(1, 4)))   # pool over time dimension

# Conv block 2
model.add(layers.Conv2D(32, (3, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(1, 4)))

# Conv block 3 (optional but helps compression)
model.add(layers.Conv2D(16, (3, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(1, 4)))

# Flatten + dense
model.add(layers.Flatten()) # 39936
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))   # binary output

model.summary()

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Simple training (you can add EarlyStopping later)
history = model.fit(
    X_train, y_train,
    epochs=100,           # start small; tune later
    batch_size=32,       # adjust according to GPU/CPU memory
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 16, 10000, 64)  │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 16, 2500, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 16, 2500, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 16, 625, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 16, 625, 16)    │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 156, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 39936)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │     2,555,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,580,753 (9.84 MB)

 Trainable params: 2,580,753 (9.84 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step - accuracy: 0.8255 - loss: 0.4749 - val_accuracy: 0.8304 - val_loss: 0.4769
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 170s 2s/step - accuracy: 0.8341 - loss: 0.4601 - val_accuracy: 0.8304 - val_loss: 0.4594
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.8341 - loss: 0.4564 - val_accuracy: 0.8304 - val_loss: 0.4619
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 170s 2s/step - accuracy: 0.8341 - loss: 0.4541 - val_accuracy: 0.8304 - val_loss: 0.4555
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.8341 - loss: 0.4538 - val_accuracy: 0.8304 - val_loss: 0.4587
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 0.8341 - loss: 0.4532 - val_accuracy: 0.8304 - val_loss: 0.4566
Epoch 7/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step - accuracy: 0.8341 - loss: 0.4516 - val_accuracy: 0.8304 - val_loss: 0.4555
Epoch 8/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step - accuracy: 0.8341 - loss: 0.4513 - val_accuracy: 0.

In [18]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([ 560, 2800]))

In [45]:
baseline_accuracy = 2800 / (2800 + 560)
baseline_accuracy

0.8333333333333334

In [33]:
model.evaluate(X_test, y_test)

45/45 ━━━━━━━━━━━━━━━━━━━━ 11s 234ms/step - accuracy: 0.8333 - loss: 0.4509


[0.450884073972702, 0.8333333134651184]

In [38]:
X_train.shape

(3360, 16, 10000, 1)

In [42]:
emg_df.head(1)

,participant,day,block,trial_id,position,grasp,signal,signal_fixed
0,1,1,1,1,2,3,"[[3.763498e-05, 1.9842508e-05], [9.071698e-06,...","[[3.763498e-05, 1.9842508e-05, 0.0, 0.0, 0.0, ..."


In [24]:

REST_GRASP_ID = 6  # confirm from README

# Build X
X = np.stack(emg_df['signal_fixed'].values)   # (N, 16, 10000)
X = np.expand_dims(X, axis=-1)                # (N, 16, 10000, 1)

# Build binary labels from grasp
grasp_labels = emg_df['grasp'].values
y = (grasp_labels != REST_GRASP_ID).astype(int)  # 0 = rest, 1 = non-rest

print("Unique y + counts:", np.unique(y, return_counts=True))
# expect something like: (array([0,1]), array([560, 2800]))

Unique y + counts: (array([0, 1]), array([ 800, 4000]))


In [25]:
indices = np.arange(len(emg_df))

train_idx, test_idx = train_test_split(
    indices,
    test_size=0.30,
    stratify=grasp_labels,   # stratify on 6-way grasp, not binary y
    random_state=42
)

X_train = X[train_idx]
X_test  = X[test_idx]
y_train = y[train_idx]
y_test  = y[test_idx]

print("Train y distribution:", np.unique(y_train, return_counts=True))
print("Test  y distribution:", np.unique(y_test, return_counts=True))

Train y distribution: (array([0, 1]), array([ 560, 2800]))
Test  y distribution: (array([0, 1]), array([ 240, 1200]))


Compute class weights so 'rest' maters more (to prevent the model always predicting 'non-rest')

In [49]:
from sklearn.utils.class_weight import compute_class_weight

classes = np.array([0, 1])
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print("Class weights:", class_weight_dict)


Class weights: {0: 3.0, 1: 0.6}


^^ errors on rest are 5× more costly than errors on active

In [50]:
model = Sequential()
model.add(layers.Input(shape=input_shape))

model.add(layers.Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(1, 4)))

model.add(layers.Conv2D(32, (3, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(1, 4)))

model.add(layers.Conv2D(16, (3, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(1, 4)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es],
    class_weight=class_weight_dict,   # 👈 make rest count more
    verbose=1
)


Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 170s 2s/step - accuracy: 0.7143 - loss: 0.6927 - val_accuracy: 0.8304 - val_loss: 0.6914
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.8341 - loss: 0.6919 - val_accuracy: 0.8304 - val_loss: 0.6923
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.8341 - loss: 0.6920 - val_accuracy: 0.8304 - val_loss: 0.6912
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.8341 - loss: 0.6919 - val_accuracy: 0.8304 - val_loss: 0.6920
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.8341 - loss: 0.6919 - val_accuracy: 0.8304 - val_loss: 0.6916


only tell you:

“about 83% of test labels are active”, and

“the model is roughly matching that distribution”.

They do not tell you:

- how many rest trials it catches,

- how many false YES decisions it makes.

That’s what matters for your BCI gate.

In [55]:
from sklearn.metrics import confusion_matrix, classification_report, recall_score
import numpy as np

# Predictions on test set
y_pred_prob = model.predict(X_test).ravel()
y_pred = (y_pred_prob >= 0.5).astype(int)

print("Confusion matrix (rows=true, cols=pred):")
cm = confusion_matrix(y_test, y_pred)
print(cm)

print("\nClassification report:")
print(classification_report(
    y_test,
    y_pred,
    target_names=['rest (no)', 'active (yes)']
))

TN, FP, FN, TP = cm.ravel()
rest_recall   = TN / (TN + FP)        # how many rest trials we correctly say NO
active_recall = TP / (TP + FN)        # how many active trials we correctly say YES

print("\nRest recall (NO):   ", rest_recall)
print("Active recall (YES):", active_recall)
print("Mean predicted label (should be < 1 if it learned NO at all):",
      y_pred.mean())


2205/2205 ━━━━━━━━━━━━━━━━━━━━ 55s 25ms/step
Confusion matrix (rows=true, cols=pred):
[[  197 12230]
 [  200 57933]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.50      0.02      0.03     12427
active (yes)       0.83      1.00      0.90     58133

    accuracy                           0.82     70560
   macro avg       0.66      0.51      0.47     70560
weighted avg       0.77      0.82      0.75     70560


Rest recall (NO):    0.015852579061720447
Active recall (YES): 0.996559613300535
Mean predicted label (should be < 1 if it learned NO at all): 0.99437358276644


In [52]:
y_pred_dummy = np.ones_like(y_test)
print("Dummy confusion matrix:")
print(confusion_matrix(y_test, y_pred_dummy))


Dummy confusion matrix:
[[   0  240]
 [   0 1200]]


after evaluating the model:


- Accuracy	-> general correctness	-> 0.83	Misleading → inflated by imbalance
- Active recall (YES) -> finds intended grasps -> 1.0 -> But trivial because  model ALWAYS predicts YES 
- Rest recall (NO) -> prevents unintended commands -> 0.0 ❌ -> Completely failing the safety requirement

A good EMG gating model would rather reject too many YES
than accidentally confirm a NO.

Right now, your model is “trigger-happy” YES.

That’s the opposite of what BCI needs.

This is a normal first result with raw EMG trials:

- Model sees strong signal → interprets everything as “movement”
- Class imbalance encourages always-YES behavior
- EMG rest looks noisy — often similar to weak activation
- No filtering, no windowing, no temporal cues → very difficult task
- Position variability adds more noise → rest becomes harder to distinguish

***POI***

To get ANY non-zero rest recall:
- Change decision threshold (0.5 → ~0.7)
- Stronger class weighting
- Normalize EMG signals channel-wise (they’re currently raw floating ADC units)
- Use windowed CNN instead of full 5s signals
- Beat the position shift with domain-generalization strategies



Immediate tweak: Adjust threshold to force some NO predictions

In [26]:
threshold = 0.7  # more conservative YES
y_pred = (y_pred_prob >= threshold).astype(int)

print(confusion_matrix(y_test, y_pred))

# We tune threshold until we see some TNs (rest correctly detected).

# This will drop accuracy but increase rest recall, which is more important


NameError: name 'y_pred_prob' is not defined

In [63]:
# Increase class weight for rest further
# class_weight_dict = {0: 8.0, 1: 1.0} -> try this if recall is still low
class_weight_dict = {0: 10.0, 1: 1.0} #-> errors for rest are 10× more costly than errors on active


In [ ]:
# preprocesss notes:

# 1. Normalize each channel

# 2. Window into 128 ms segments (paper standard)

# 3. Label windows inside rest trials → uniform rest patterns

# 4. CNN learns short-term EMG bursts, not full 5s

Second model after evaluating first model

In [27]:
# EMG array: (n_trials, 16, 10000)
X = np.stack(emg_df['signal_fixed'].values)

grasp_labels = emg_df['grasp'].values
REST_GRASP_ID = 6
y = (grasp_labels != REST_GRASP_ID).astype(int)  # 0 = rest, 1 = active

print("y distribution:", np.unique(y, return_counts=True))


y distribution: (array([0, 1]), array([ 800, 4000]))


In [28]:
# Stratified split (on 6-grasp labels)

# split by indices so to reuse the same split after normalization etc.

indices = np.arange(len(emg_df))

train_idx, test_idx = train_test_split(
    indices,
    test_size=0.30,
    stratify=grasp_labels,   # stratify by 6-way grasp
    random_state=42
)

X_train_raw = X[train_idx]   # (n_train, 16, 10000)
X_test_raw  = X[test_idx]

y_train = y[train_idx]
y_test  = y[test_idx]

print("Train y:", np.unique(y_train, return_counts=True))
print("Test  y:", np.unique(y_test, return_counts=True))

Train y: (array([0, 1]), array([ 560, 2800]))
Test  y: (array([0, 1]), array([ 240, 1200]))


In [29]:
# NORMALIZATION - Compute mean & std per channel over all time + trials in TRAIN
train_mean = X_train_raw.mean(axis=(0, 2), keepdims=True)  # (1, 16, 1)
train_std  = X_train_raw.std(axis=(0, 2), keepdims=True) + 1e-8

X_train_norm = (X_train_raw - train_mean) / train_std
X_test_norm  = (X_test_raw  - train_mean) / train_std  # use train stats!

# Add channel dimension for Conv2D: (batch, 16, 10000, 1)
X_train = np.expand_dims(X_train_norm, axis=-1)
X_test  = np.expand_dims(X_test_norm,  axis=-1)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (3360, 16, 10000, 1)
X_test shape: (1440, 16, 10000, 1)


In [53]:
# evaluation function with recall
def evaluate_gate_model(model, X_test, y_test, threshold=0.5):
    y_pred_prob = model.predict(X_test).ravel()
    y_pred = (y_pred_prob >= threshold).astype(int)

    cm = confusion_matrix(y_test, y_pred)
    TN, FP, FN, TP = cm.ravel()

    rest_recall   = TN / (TN + FP) if (TN + FP) > 0 else 0.0
    active_recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0

    print(f"\n=== Threshold {threshold:.2f} ===")
    print("Confusion matrix (rows=true, cols=pred):")
    print(cm)

    print("\nClassification report:")
    print(classification_report(
        y_test, y_pred,
        target_names=['rest (no)', 'active (yes)']
    ))

    print("Rest recall (NO):   ", rest_recall)
    print("Active recall (YES):", active_recall)
    print("Mean predicted label:", y_pred.mean())

    return cm, rest_recall, active_recall


In [ ]:
# Stronger class weights (make rest “expensive” by aggresively weighting class 0)
# i.e. error for rest peanlised 10x more than active grasps

class_weight_dict = {0: 10.0, 1: 1.0}
print("Class weights:", class_weight_dict)

Class weights: {0: 10.0, 1: 1.0}


In [ ]:
input_shape = X_train.shape[1:]   # (16, 10000, 1)

model = Sequential()
model.add(layers.Input(shape=input_shape))

model.add(layers.Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(1, 4)))

model.add(layers.Conv2D(32, (3, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(1, 4)))

model.add(layers.Conv2D(16, (3, 3), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(1, 4)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', 'precision', 'recall']  # include Recall metric
)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es],              # your EarlyStopping callback
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.6369 - loss: 1.1313 - val_accuracy: 0.7351 - val_loss: 0.5056
Epoch 2/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 182s 2s/step - accuracy: 0.7533 - loss: 0.8755 - val_accuracy: 0.8036 - val_loss: 0.4374
Epoch 3/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 188s 2s/step - accuracy: 0.7719 - loss: 0.8188 - val_accuracy: 0.8661 - val_loss: 0.3000
Epoch 4/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.8028 - loss: 0.7283 - val_accuracy: 0.7455 - val_loss: 0.6324
Epoch 5/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.8006 - loss: 0.7250 - val_accuracy: 0.7738 - val_loss: 0.5252
Epoch 6/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step - accuracy: 0.8110 - loss: 0.6831 - val_accuracy: 0.9062 - val_loss: 0.2147
Epoch 7/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.8181 - loss: 0.6597 - val_accuracy: 0.8661 - val_loss: 0.2975
Epoch 8/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.8170 - loss: 0.6816 - val_accuracy: 0.8363 - v

In [79]:
# Evaluate at multiple thresholds - First at the usual 0.5:

evaluate_gate_model(model, X_test, y_test, threshold=0.5)


45/45 ━━━━━━━━━━━━━━━━━━━━ 11s 224ms/step

=== Threshold 0.50 ===
Confusion matrix (rows=true, cols=pred):
[[ 194   46]
 [ 110 1090]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.64      0.81      0.71       240
active (yes)       0.96      0.91      0.93      1200

    accuracy                           0.89      1440
   macro avg       0.80      0.86      0.82      1440
weighted avg       0.91      0.89      0.90      1440

Rest recall (NO):    0.8083333333333333
Active recall (YES): 0.9083333333333333
Mean predicted label: 0.7888888888888889


(array([[ 194,   46],
        [ 110, 1090]]),
 0.8083333333333333,
 0.9083333333333333)

In [81]:
# Then sweep a few thresholds to see tradeoffs:
for th in [0.3, 0.4, 0.5, 0.6, 0.7]:
    evaluate_gate_model(model, X_test, y_test, threshold=th)


45/45 ━━━━━━━━━━━━━━━━━━━━ 11s 220ms/step

=== Threshold 0.30 ===
Confusion matrix (rows=true, cols=pred):
[[ 147   93]
 [  62 1138]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.70      0.61      0.65       240
active (yes)       0.92      0.95      0.94      1200

    accuracy                           0.89      1440
   macro avg       0.81      0.78      0.80      1440
weighted avg       0.89      0.89      0.89      1440

Rest recall (NO):    0.6125
Active recall (YES): 0.9483333333333334
Mean predicted label: 0.8548611111111111
45/45 ━━━━━━━━━━━━━━━━━━━━ 11s 234ms/step

=== Threshold 0.40 ===
Confusion matrix (rows=true, cols=pred):
[[ 179   61]
 [  83 1117]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.68      0.75      0.71       240
active (yes)       0.95      0.93      0.94      1200

    accuracy                           0.90      1440
   macro avg       0.82      0

*70% threshold has the best recall*

- rest recall (NO):   0.89  ← Safety metric: excellent
- active recall (YES): 0.86  ← Efficiency metric: excellent
- accuracy:            0.86  ← Higher than baseline 0.83
- mean predicted label: 0.73 ← Model predicts YES ~73% of time


NEXT STEP - WINDOWING CNN

why?

| Full 5s CNN (current)               | Windowed CNN (next)              |
| ----------------------------------- | -------------------------------- |
| Very long input → harder learning   | Shorter patterns easier to learn |
| Ignores temporal dynamics           | Learns EMG bursts in time        |
| Harder to generalize to new motions | More robust and fast             |
| Latency = whole trial               | Real-time sub-second decisions   |


Windowed CNN will:

improve both recalls

reduce latency dramatically (needed for fluent BCI typing)

support sliding real-time control instead of full trial prediction
___
🔮 Future goal

Predict YES/NO within 200ms windows during live EMG streaming

------

^Summary of where the modelling stands^


| Component                       | Status                         |
| ------------------------------- | ------------------------------ |
| Binary classification working   | ✔                              |
| Rest recall high                | ✔ 0.89 (fantastic for safety!) |
| Active recall still high        | ✔ 0.86 (efficient)             |
| Balanced safety & usability     | ✔                              |
| Architecture baseline done      | ✔                              |
| Threshold decision rule correct | ✔                              |


Next phase: Windowed CNN

1. Segment each trial into windows (e.g., 200 ms = 400 samples)
2. Normalize window signals
3. Train same CNN architecture on windows
4. Use voting or averaged probability for trial-level output

helper recomendation: 200 ms windows with 50% overlap (step = 100 ms / 200 samples) -> sweet spot for usability

In [ ]:
# window params: 200 ms window length
# Sampling rate 2000 Hz → 200 ms = 400 samples
# 50% overlap → stride = 200 samples

In [30]:
def make_windows(X, y, win_len=400, step=200):
    """
    X: (n_trials, 16, T)
    y: trial-level labels (0/1)
    """
    X_w, y_w, trial_ids = [], [], []
    for i, trial in enumerate(X):
        T = trial.shape[1]
        for start in range(0, T-win_len+1, step):
            end = start + win_len
            window = trial[:, start:end]  # (16, win_len)
            X_w.append(window)
            y_w.append(y[i])
            trial_ids.append(i)
    return np.stack(X_w), np.array(y_w), np.array(trial_ids)

# Build windows
X_windows_raw, y_windows, window2trial = make_windows(X, y)

print("X_windows_raw:", X_windows_raw.shape)
print("y_windows:", y_windows.shape)
print("Unique trial mapping:", len(np.unique(window2trial)))


X_windows_raw: (235200, 16, 400)
y_windows: (235200,)
Unique trial mapping: 4800


In [ ]:
# Train/test split on windows (not trials!)
win_idx = np.arange(len(X_windows_raw))

train_w_idx, test_w_idx = train_test_split(
    win_idx,
    test_size=0.30,
    stratify=y_windows,
    random_state=42
)

X_train_win_raw = X_windows_raw[train_w_idx]
X_test_win_raw  = X_windows_raw[test_w_idx]

y_train_win = y_windows[train_w_idx]
y_test_win  = y_windows[test_w_idx]

print("Window-level y dist train:", np.unique(y_train_win, return_counts=True))

Window-level y dist train: (array([0, 1]), array([ 27440, 137200]))


In [ ]:
#Normalize window inputs
train_mean = X_train_win_raw.mean(axis=(0, 2), keepdims=True)
train_std  = X_train_win_raw.std(axis=(0, 2), keepdims=True) + 1e-8

X_train_win = (X_train_win_raw - train_mean) / train_std
X_test_win  = (X_test_win_raw  - train_mean) / train_std

# TensorFlow shape: (batch, 16, 400, 1)
X_train_win = np.expand_dims(X_train_win, axis=-1)
X_test_win  = np.expand_dims(X_test_win, axis=-1)


In [89]:
# Strong class weights (same as before)
class_weight_win = {0: 10.0, 1: 1.0}


In [90]:
# Build + train the SAME CNN but on windows
input_shape = X_train_win.shape[1:]  # (16, 400, 1)

model_win = Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1,4)),
    layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1,4)),
    layers.Conv2D(16, (3,3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1,4)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_win.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history_win = model_win.fit(
    X_train_win, y_train_win,
    epochs=25,
    batch_size=64,
    validation_split=0.2,
    class_weight=class_weight_win,
    callbacks=[es],
    verbose=1
)


Epoch 1/25
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 171s 83ms/step - accuracy: 0.1769 - loss: 1.5893 - val_accuracy: 0.1775 - val_loss: 1.0005
Epoch 2/25
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 166s 81ms/step - accuracy: 0.1780 - loss: 1.5801 - val_accuracy: 0.1802 - val_loss: 0.9757
Epoch 3/25
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 165s 80ms/step - accuracy: 0.1784 - loss: 1.5786 - val_accuracy: 0.1806 - val_loss: 0.9673
Epoch 4/25
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 165s 80ms/step - accuracy: 0.1786 - loss: 1.5781 - val_accuracy: 0.1780 - val_loss: 0.9863
Epoch 5/25
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 168s 82ms/step - accuracy: 0.1790 - loss: 1.5764 - val_accuracy: 0.1801 - val_loss: 0.9786


^^^^^^
This simulates real-time performance per trial AND Should outperform full-trial CNN by a large margin

This^ 3rd itterated model should:
- train MUCH faster and learn richer features
- Recall for rest/no should greatly improve

In [91]:
# # Window-level evaluation
evaluate_gate_model(model_win, X_test_win, y_test_win, threshold=0.7)


2205/2205 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step

=== Threshold 0.70 ===
Confusion matrix (rows=true, cols=pred):
[[11754     6]
 [58188   612]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.17      1.00      0.29     11760
active (yes)       0.99      0.01      0.02     58800

    accuracy                           0.18     70560
   macro avg       0.58      0.50      0.15     70560
weighted avg       0.85      0.18      0.07     70560

Rest recall (NO):    0.9994897959183674
Active recall (YES): 0.010408163265306122
Mean predicted label: 0.008758503401360545


(array([[11754,     6],
        [58188,   612]]),
 0.9994897959183674,
 0.010408163265306122)

In [92]:
# Trial-level evaluation via majority vote
# Predict on ALL windows
y_prob_all = model_win.predict(np.expand_dims(X_windows_raw, -1)).ravel()
y_pred_all = (y_prob_all >= 0.7).astype(int)  # threshold tuned later

# Aggregate per trial
trial_pred = {}
for pred, tid in zip(y_pred_all, window2trial):
    trial_pred.setdefault(tid, []).append(pred)

trial_pred_final = np.array([
    int(np.mean(preds) >= 0.5)  # vote YES if majority windows = YES
    for tid, preds in sorted(trial_pred.items())
])

print("Trial-level confusion:")
cm = confusion_matrix(y, trial_pred_final)
print(cm)

TN, FP, FN, TP = cm.ravel()
print("\nTrial-level Rest recall:", TN/(TN+FP))
print("Trial-level Active recall:", TP/(TP+FN))


7350/7350 ━━━━━━━━━━━━━━━━━━━━ 80s 11ms/step
Trial-level confusion:
[[ 800    0]
 [4000    0]]

Trial-level Rest recall: 1.0
Trial-level Active recall: 0.0


This version of the EMG model is extremely safe — it almost never triggers YES accidentally.
But right now it's too cautious, and barely ever confirms actual intended movements.
In other words: great at rejecting mistakes, but too strict to be practical.

Next step is to adjust the balance so the user can reliably confirm intended commands while still preventing incorrect ones.**

In [93]:
# We’re building the EMG model as a YES/NO gate to confirm EEG selections.
# Right now the model is excellent at rejecting unintended activations,
# and now we’re tuning it so it also confirms intended activations reliably.
# The architecture is correct — we’re just calibrating the decision balance.”

Once you run these:

1️⃣ Window-level metrics

2️⃣ Trial-level metrics

--

Post the confusion matrix + recalls and we’ll:

- adjust the ideal threshold

- consider adding more robust CNN architecture

- consider cross-arm-position generalization (harder version)

- prepare packaging functions for MLflow deployment

to calibrate the decision balance:
1. Reduce class weight on rest to {0:5, 1:1} and retune threshold closer to 0.6

Goal:

Increase active recall (YES confirmations)

Keep rest recall (NO safety) still above ~0.90

Avoid returning to the “always yes” or “always no” behavior

In [95]:
class_weight_win2 = {0: 5.0, 1: 1.0}
print(class_weight_win2)


{0: 5.0, 1: 1.0}


In [96]:
# Re-train the windowed CNN with new weights
model_win2 = Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1,4)),
    layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1,4)),
    layers.Conv2D(16, (3,3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1,4)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_win2.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history_win2 = model_win2.fit(
    X_train_win, y_train_win,
    epochs=25,
    batch_size=64,
    validation_split=0.2,
    class_weight=class_weight_win2,
    callbacks=[es],
    verbose=1
)


Epoch 1/25
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 181s 88ms/step - accuracy: 0.4700 - loss: 1.1533 - val_accuracy: 0.8330 - val_loss: 0.6850
Epoch 2/25
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 163s 79ms/step - accuracy: 0.4918 - loss: 1.1464 - val_accuracy: 0.8310 - val_loss: 0.6882
Epoch 3/25
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 158s 77ms/step - accuracy: 0.6751 - loss: 1.1471 - val_accuracy: 0.1801 - val_loss: 0.6912
Epoch 4/25
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 161s 78ms/step - accuracy: 0.2990 - loss: 1.1449 - val_accuracy: 0.8331 - val_loss: 0.6781
Epoch 5/25
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 160s 78ms/step - accuracy: 0.6055 - loss: 1.1476 - val_accuracy: 0.1792 - val_loss: 0.6895


In [97]:
# Evaluate at multiple thresholds between 0.5 and 0.7 (0.7 was best threshold for intial recall optimise model)
thresholds = [0.5, 0.55, 0.6, 0.65, 0.7]

for th in thresholds:
    evaluate_gate_model(model_win2, X_test_win, y_test_win, threshold=th)

# ideally:
# Rest recall (NO)	-> stays high (≥0.85)
# Active recall (YES)	-> increases significantly (≥0.50 → aiming toward 0.80+)
# Mean predicted label	-> between 0.20–0.40


2205/2205 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step

=== Threshold 0.50 ===
Confusion matrix (rows=true, cols=pred):
[[  179 11581]
 [  103 58697]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.63      0.02      0.03     11760
active (yes)       0.84      1.00      0.91     58800

    accuracy                           0.83     70560
   macro avg       0.73      0.51      0.47     70560
weighted avg       0.80      0.83      0.76     70560

Rest recall (NO):    0.015221088435374149
Active recall (YES): 0.9982482993197279
Mean predicted label: 0.9960034013605442
2205/2205 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step

=== Threshold 0.55 ===
Confusion matrix (rows=true, cols=pred):
[[11732    28]
 [57835   965]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.17      1.00      0.29     11760
active (yes)       0.97      0.02      0.03     58800

    accuracy                           0.18     70560
  

current conclusion 
model is bimodal:

- If threshold ≤ 0.50 → almost always YES
- If threshold > 0.50 → almost always NO

In [ ]:
# # If performance looks good → We evaluate trial-level voting

# # Predict on ALL windows
# y_prob_all2 = model_win2.predict(np.expand_dims(X_windows_raw, -1)).ravel()

# # Pick the best threshold from the sweep, e.g. 0.6
# best_th = 0.6
# y_pred_all2 = (y_prob_all2 >= best_th).astype(int)

# # Aggregate per trial
# trial_pred2 = {}
# for pred, tid in zip(y_pred_all2, window2trial):
#     trial_pred2.setdefault(tid, []).append(pred)

# trial_pred_final2 = np.array([
#     int(np.mean(preds) >= 0.5)
#     for tid, preds in sorted(trial_pred2.items())
# ])

# print("Trial-level confusion matrix:")
# cm = confusion_matrix(y, trial_pred_final2)
# print(cm)

# TN, FP, FN, TP = cm.ravel()
# print("Trial-level Rest recall:", TN/(TN+FP))
# print("Trial-level Active recall:", TP/(TP+FN))


What we do NOW (new plan)

We must fix window labeling + features.

Phase 3: Physiological Feature Modeling

We introduce:

- EMG Preprocessing Pipeline

- Bandpass filter 20–450 Hz

- Absolute value (rectify the signal)

- Channel normalization per window

In [31]:
# Only want to label windows during active contraction

# Can automate detection by measuring EMG energy:

# computing RMS (root mean squared) amplitude per window:

def window_features_rms(window):
    # window: (16, win_len)
    rms = np.sqrt(np.mean(window**2, axis=1))
    return rms.mean()  # average across channels


Next:
- Compute RMS for each window in a trial

- Use a percentile threshold per trial to detect peaks → contraction region

- Only label strong-activation windows as YES

- Everything else becomes NO

This will dramatically improve learnability.

In [100]:
# Window type	        Old label	         New label
# True contraction	    YES	              YES
# Early quiet part	    YES	              NO (corrected)
# Hesitation / release	YES	              NO (corrected)

Below is a full pipeline that:

- Builds windows from signal_fixed

- Uses EMG RMS energy to relabel which windows are truly “active” vs “rest”

- Normalizes the EMG

- Retrains a CNN on these refined window labels

- You can paste this into your notebook in chunks (cells).

In [32]:
#From trial-level EMG → raw windows
# Trial-level EMG
X_trials = np.stack(emg_df['signal_fixed'].values)    # (n_trials, 16, 10000)
grasp_labels = emg_df['grasp'].values
REST_GRASP_ID = 6
y_trial = (grasp_labels != REST_GRASP_ID).astype(int)  # 0 = rest, 1 = non-rest

print("Trials shape:", X_trials.shape)
print("Trial labels (0=rest,1=active):", np.unique(y_trial, return_counts=True))

Trials shape: (4800, 16, 10000)
Trial labels (0=rest,1=active): (array([0, 1]), array([ 800, 4000]))


In [33]:
# 200ms windows with 50% overlap
WIN_LEN = 400   # 200 ms @ 2000 Hz
STEP    = 200   # 50% overlap

def make_windows(X_trials, y_trials, win_len=400, step=200):
    X_w = []
    y_trial_w = []
    trial_ids = []
    for t_idx, trial in enumerate(X_trials):
        T = trial.shape[1]
        for start in range(0, T - win_len + 1, step):
            end = start + win_len
            X_w.append(trial[:, start:end])   # (16, win_len)
            y_trial_w.append(y_trials[t_idx])
            trial_ids.append(t_idx)
    return np.stack(X_w), np.array(y_trial_w), np.array(trial_ids)

X_win_raw, y_trial_win, win_trial_ids = make_windows(X_trials, y_trial, WIN_LEN, STEP)

print("Window EMG shape:", X_win_raw.shape)   # (n_windows, 16, 400)
print("Window trial labels:", np.unique(y_trial_win, return_counts=True))


Window EMG shape: (235200, 16, 400)
Window trial labels: (array([0, 1]), array([ 39200, 196000]))


In [34]:
# Compute RMS per window & relabel windows

# We’ll treat only high-energy windows in active trials as positive (1).
# Everything else becomes 0 (rest-like / no activation).

# --------
# RMS energy per window: mean over time then over channels
# X_win_raw: (n_windows, 16, 400)
rms_per_window = np.sqrt(np.mean(X_win_raw**2, axis=2)).mean(axis=1)  # (n_windows,)

# New window-level labels (0/1)
y_win_refined = np.zeros_like(y_trial_win)

for t in np.unique(win_trial_ids):
    mask = (win_trial_ids == t)
    trial_rms = rms_per_window[mask]

    if y_trial[t] == 0:
        # Rest trial → all windows are labeled rest (0)
        y_win_refined[mask] = 0
    else:
        # Active trial → only highest-energy windows are labeled 1
        # You can tune this percentile (e.g. 60, 70, 80)
        thr = np.percentile(trial_rms, 70)
        y_win_refined[mask] = (trial_rms >= thr).astype(int)

print("Refined window labels:", np.unique(y_win_refined, return_counts=True))


Refined window labels: (array([0, 1]), array([ 39200, 196000]))


Instead of saying “all windows in an active trial are 1”, we now say:

- windows with strong EMG bursts → 1

- everything else → 0

In [35]:
# Train/test split at window level + normalization
win_indices = np.arange(len(X_win_raw))

train_idx, test_idx = train_test_split(
    win_indices,
    test_size=0.30,
    stratify=y_win_refined,
    random_state=42
)

X_train_raw = X_win_raw[train_idx]
X_test_raw  = X_win_raw[test_idx]
y_train = y_win_refined[train_idx]
y_test  = y_win_refined[test_idx]

print("Train labels:", np.unique(y_train, return_counts=True))
print("Test labels:", np.unique(y_test, return_counts=True))

Train labels: (array([0, 1]), array([ 27440, 137200]))
Test labels: (array([0, 1]), array([11760, 58800]))


In [36]:
# Normalize using training windows only

# -----
# Channel-wise normalization (over all train windows & time)
train_mean = X_train_raw.mean(axis=(0, 2), keepdims=True)  # (1, 16, 1)
train_std  = X_train_raw.std(axis=(0, 2), keepdims=True) + 1e-8

X_train = (X_train_raw - train_mean) / train_std
X_test  = (X_test_raw  - train_mean) / train_std

# Add channel dimension for Conv2D: (batch, 16, 400, 1)
X_train = X_train[..., np.newaxis]
X_test  = X_test[..., np.newaxis]

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (164640, 16, 400, 1)
X_test shape: (70560, 16, 400, 1)


In [38]:
#Class weights from refined window labels

from sklearn.utils.class_weight import compute_class_weight

classes = np.array([0, 1])
weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weight = {0: float(weights[0]), 1: float(weights[1])}
print("Class weights:", class_weight)

Class weights: {0: 3.0, 1: 0.6}


In [50]:
es = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [110]:
input_shape = X_train.shape[1:]   # (16, 400, 1)

model_win = Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1, 2)),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1, 2)),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_win.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', 'precision', 'recall']
)

history_win = model_win.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    class_weight=class_weight,
    callbacks=[es],
    verbose=1
)

Epoch 1/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 387s 188ms/step - accuracy: 0.4073 - loss: 0.6894 - precision: 0.8405 - recall: 0.3568 - val_accuracy: 0.8320 - val_loss: 0.6816 - val_precision: 0.8345 - val_recall: 0.9956
Epoch 2/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 382s 186ms/step - accuracy: 0.5542 - loss: 0.6867 - precision: 0.8359 - recall: 0.5788 - val_accuracy: 0.8299 - val_loss: 0.6860 - val_precision: 0.8341 - val_recall: 0.9932
Epoch 3/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 383s 186ms/step - accuracy: 0.5284 - loss: 0.6862 - precision: 0.8382 - recall: 0.5382 - val_accuracy: 0.1811 - val_loss: 0.6875 - val_precision: 0.9069 - val_recall: 0.0178
Epoch 4/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 382s 186ms/step - accuracy: 0.4723 - loss: 0.6854 - precision: 0.8396 - recall: 0.4537 - val_accuracy: 0.8329 - val_loss: 0.6813 - val_precision: 0.8345 - val_recall: 0.9970
Epoch 5/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 420s 204ms/step - accuracy: 0.4968 - loss: 0.6850 - precision: 0.8375 - recall: 0.4918 - val_a

In [111]:
# Try a few thresholds again
for th in [0.4, 0.5, 0.6]:
    evaluate_gate_model(model_win, X_test, y_test, threshold=th)


2205/2205 ━━━━━━━━━━━━━━━━━━━━ 57s 26ms/step

=== Threshold 0.40 ===
Confusion matrix (rows=true, cols=pred):
[[  156 11604]
 [   67 58733]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.70      0.01      0.03     11760
active (yes)       0.84      1.00      0.91     58800

    accuracy                           0.83     70560
   macro avg       0.77      0.51      0.47     70560
weighted avg       0.81      0.83      0.76     70560

Rest recall (NO):    0.013265306122448979
Active recall (YES): 0.9988605442176871
Mean predicted label: 0.9968395691609977
2205/2205 ━━━━━━━━━━━━━━━━━━━━ 55s 25ms/step

=== Threshold 0.50 ===
Confusion matrix (rows=true, cols=pred):
[[  171 11589]
 [   81 58719]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.68      0.01      0.03     11760
active (yes)       0.84      1.00      0.91     58800

    accuracy                           0.83     70560
  

In [112]:
# What does this output mean:

# the model is STILL not distinguishing between rest EMG and active EMG windows!

# The EMG bursts are too subtle for the model to detect in its current form.
# We need stronger signal-processing features before training the classifier.


Phase 3 — Real EMG Signal Processing + Better Activation Labels

processing steps:
- Bandpass filter (20–450 Hz)	-> remove motion artifacts + high-freq noise
- Rectify (abs)	-> convert EMG to magnitude signal
- RMS envelope (downsampled)	-> extract muscle activation bursts
- Peak-based window relabeling	-> only windows with real grips get YES label

In [39]:
# Bandpass filter + full EMG preprocessing
from scipy.signal import butter, filtfilt

FS = 2000  # sampling rate (Hz)
LOWCUT = 20
HIGHCUT = 450

def bandpass_filter(signal, lowcut=LOWCUT, highcut=HIGHCUT, fs=FS, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

def preprocess_emg(trial):
    """
    trial: (16, 10000)
    returns same shape, preprocessed
    """
    filtered = np.zeros_like(trial)
    for ch in range(trial.shape[0]):
        filtered[ch] = bandpass_filter(trial[ch])
    rectified = np.abs(filtered)
    return rectified


In [41]:
# apply preprocessing
X_trials_proc = np.array([preprocess_emg(tr) for tr in X_trials])
print("Processed trials shape:", X_trials_proc.shape)


Processed trials shape: (4800, 16, 10000)


In [42]:
# create windows (same shape as before)
X_win, y_trial_win, win_trial_ids = make_windows(X_trials_proc, y_trial, WIN_LEN, STEP)
print("Windows:", X_win.shape)

Windows: (235200, 16, 400)


In [43]:
#compute RMS enevlope per window
# ----
# RMS across time & channels
rms_win = np.sqrt(np.mean(X_win**2, axis=2)).mean(axis=1)  # (n_windows,)

In [ ]:
# visualize range
print("RMS stats:", np.min(rms_win), np.max(rms_win), np.mean(rms_win))
#?

RMS stats: 0.0 1.0901903e-05 4.043176e-08


Relabel windows based on real activation peaks

Instead of top 30%, adapt threshold per trial

In [44]:
y_win = np.zeros_like(y_trial_win)

for t in np.unique(win_trial_ids):
    mask = (win_trial_ids == t)
    trial_rms = rms_win[mask]

    if y_trial[t] == 1:  # active trial
        thr = np.percentile(trial_rms, 80)  # ✓ only top 20% windows are grip effort
        y_win[mask] = (trial_rms >= thr).astype(int)
    else:
        y_win[mask] = 0


In [45]:
# Split windows + Normalize

win_idx = np.arange(len(X_win))
train_idx, test_idx = train_test_split(
    win_idx, test_size=0.30,
    stratify=y_win, random_state=42
)

X_train_raw = X_win[train_idx]
X_test_raw  = X_win[test_idx]
y_train = y_win[train_idx]
y_test  = y_win[test_idx]

train_mean = X_train_raw.mean(axis=(0, 2), keepdims=True)
train_std = X_train_raw.std(axis=(0, 2), keepdims=True) + 1e-8

X_train = (X_train_raw - train_mean) / train_std
X_test  = (X_test_raw  - train_mean) / train_std

X_train = X_train[..., np.newaxis]
X_test  = X_test[..., np.newaxis]

print("Train shapes:", X_train.shape, y_train.shape)


Train shapes: (164640, 16, 400, 1) (164640,)


In [46]:
# Compute class weights automatically
from sklearn.utils.class_weight import compute_class_weight

classes = np.array([0,1])
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight = {0: float(weights[0]), 1: float(weights[1])}

print("Class weights:", class_weight)

Class weights: {0: 2.839012277555525, 1: 0.6068827224204535}


In [ ]:
# checkpoint -> callbacks.ModelCheckpoint() to save best model during training.
# after final model is completed, have a 'export' model function to export as a pickle or h5 file.
# balance the ratio of classes (make it equal representation of rest and active) by oversampling the minority class or undersampling the majority class.

In [ ]:
#Train the CNN

input_shape = X_train.shape[1:]

model = Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1,2)),
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1,2)),
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.MaxPool2D(pool_size=(1,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    class_weight=class_weight,
    callbacks=[es],
    verbose=1
)


Epoch 1/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 399s 194ms/step - accuracy: 0.4510 - loss: 0.6919 - val_accuracy: 0.1889 - val_loss: 0.6951
Epoch 2/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 391s 190ms/step - accuracy: 0.3794 - loss: 0.6880 - val_accuracy: 0.8228 - val_loss: 0.6802
Epoch 3/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 399s 194ms/step - accuracy: 0.6811 - loss: 0.6869 - val_accuracy: 0.1907 - val_loss: 0.6939
Epoch 4/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 390s 190ms/step - accuracy: 0.4063 - loss: 0.6865 - val_accuracy: 0.1918 - val_loss: 0.6913
Epoch 5/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 392s 190ms/step - accuracy: 0.4375 - loss: 0.6867 - val_accuracy: 0.8242 - val_loss: 0.6826
Epoch 6/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 397s 193ms/step - accuracy: 0.4394 - loss: 0.6874 - val_accuracy: 0.8226 - val_loss: 0.6812
Epoch 7/30
2058/2058 ━━━━━━━━━━━━━━━━━━━━ 403s 196ms/step - accuracy: 0.5489 - loss: 0.6863 - val_accuracy: 0.8227 - val_loss: 0.6844


In [56]:
# Evaluate with threshold sweep
for th in [0.4, 0.5, 0.6]:
    evaluate_gate_model(model, X_test, y_test, threshold=th)


2205/2205 ━━━━━━━━━━━━━━━━━━━━ 55s 25ms/step

=== Threshold 0.40 ===
Confusion matrix (rows=true, cols=pred):
[[  182 12245]
 [  167 57966]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.52      0.01      0.03     12427
active (yes)       0.83      1.00      0.90     58133

    accuracy                           0.82     70560
   macro avg       0.67      0.51      0.47     70560
weighted avg       0.77      0.82      0.75     70560

Rest recall (NO):    0.014645529894584373
Active recall (YES): 0.9971272771059467
Mean predicted label: 0.9950538548752834
2205/2205 ━━━━━━━━━━━━━━━━━━━━ 55s 25ms/step

=== Threshold 0.50 ===
Confusion matrix (rows=true, cols=pred):
[[  197 12230]
 [  200 57933]]

Classification report:
              precision    recall  f1-score   support

   rest (no)       0.50      0.02      0.03     12427
active (yes)       0.83      1.00      0.90     58133

    accuracy                           0.82     70560
  

In [59]:
emg_df['grasp'].value_counts()

grasp
3    800
1    800
4    800
5    800
2    800
6    800
Name: count, dtype: int64